In [1]:
import  numpy as np
import pandas as pd
import ast
import nltk
import pickle as pk

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
credits.rename(columns = {'movie_id' : 'id'}, inplace=True)

In [4]:
movies = movies.merge(credits, on='id')

In [5]:
movies = movies[['id','original_title','genres','keywords','overview','production_companies','cast','crew','release_date','runtime','homepage']]

In [6]:
def conv(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [7]:
movies['genres'] = movies['genres'].apply(conv)

In [8]:
movies['keywords'] = movies['keywords'].apply(conv)

In [9]:
movies['production_companies'] = movies['production_companies'].apply(conv)

In [10]:
def convert(obj):
    L=[]
    c=0
    for i in ast.literal_eval(obj):
        if c!=5:
            L.append(i['name'])
            c+=1
        else:
            break
    return L

In [11]:
movies['cast'] = movies['cast'].apply(convert)

In [12]:
movies['producer'] = movies['crew']

In [13]:
movies.rename(columns = {'crew' : 'director'}, inplace=True)

In [14]:
def dp(obj,job):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']==job:
            L.append(i['name'])
            break
    return L

In [15]:
movies['director'] = movies['director'].apply(lambda x:dp(x,'Director'))

In [16]:
movies['producer'] = movies['producer'].apply(lambda x:dp(x,'Producer'))

In [17]:
movies["tag"] = movies['overview'].apply(lambda x:str(x).split()) + movies['genres'].apply(lambda x:[i.replace(" ","") for i in x]) + movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x]) + movies['production_companies'].apply(lambda x:[i.replace(" ","") for i in x]) + movies['cast'].apply(lambda x:[i.replace(" ","") for i in x]) + movies['director'].apply(lambda x:[i.replace(" ","") for i in x]) + movies['producer'].apply(lambda x:[i.replace(" ","") for i in x])

In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [19]:
movies['tag'] = movies['tag'].apply(lambda x:[ps.stem(i) for i in x])

In [21]:
movies['tag'] = movies['tag'].apply(lambda x:' '.join(x))

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2*len(movies), stop_words='english')

In [25]:
vec = cv.fit_transform(movies['tag']).toarray()

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
sim = cosine_similarity(vec)

In [28]:
def recommend(mov,count):
    a = 6 + count*5
    mov_index = movies[movies['original_title'] == mov].index[0]
    mov_list = sorted(list(enumerate(sim[mov_index])),reverse=True,key=lambda x:x[1])[1:a]
    
    for i in mov_list:
        print(movies['original_title'][i[0]])

In [40]:
recommend("Batman",0)

Batman & Robin
Batman Returns
Batman Begins
The R.M.
Batman Forever


In [ ]:
pk.dump()